In [1]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-9e51b6ad-1e37-4393-bbf0-54836c7c6bbc',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'UA8jNeMWxBUgJy32OfWxtJsa1bk7yOSOrpgYAYuL4xMu'
}

configuration_name = 'os_56fd1e2bb7b94877a7f99dabe7a4ba45_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face issues with the DataFrame layout.
# Please read the documentation of 'SparkSession.read()' to learn more about the possibilities to adjust the data loading.
# PySpark documentation: http://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrameReader.json

df = spark.read.json(cos.url('medium-sparkify-event-data.json', 'sparkify-donotdelete-pr-qz53avfoxtdc2u'))
df.take(5)


Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200427050600-0000
KERNEL_ID = cfc09f94-e2f1-42d5-b5ad-4a53f3b61d3c


[Row(artist='Martin Orford', auth='Logged In', firstName='Joseph', gender='M', itemInSession=20, lastName='Morales', length=597.55057, level='free', location='Corpus Christi, TX', method='PUT', page='NextSong', registration=1532063507000, sessionId=292, song='Grand Designs', status=200, ts=1538352011000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='293'),
 Row(artist="John Brown's Body", auth='Logged In', firstName='Sawyer', gender='M', itemInSession=74, lastName='Larson', length=380.21179, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='PUT', page='NextSong', registration=1538069638000, sessionId=97, song='Bulls', status=200, ts=1538352025000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='98'),
 Row(artist='Afroman', auth='Logged In', firstName='Maverick', gender='M', 

In [2]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer, MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier
from sklearn.metrics import confusion_matrix
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
spark = SparkSession \
    .builder \
    .appName("Sparkify Project") \
    .getOrCreate()

In [4]:
df = spark.read.json("mini_sparkify_event_data.json")

AnalysisException: 'Path does not exist: file:/home/spark/shared/mini_sparkify_event_data.json;'

In [ ]:
df.take(5)

In [ ]:
df.printSchema()

In [ ]:
user_log.describe().toPandas()

In [ ]:
user_log.count()

In [ ]:
print(len(user_log.columns))

In [ ]:
user_log.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in user_log.columns]).toPandas()

In [ ]:
user_log.groupby('page').count().toPandas()

In [ ]:
df = user_log.dropna(how = "any", subset = ["userId", "sessionId"])


df.count()

In [ ]:
df = df.filter(df["userId"] != "")
df.count()

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).toPandas()

In [ ]:
df.select("page").dropDuplicates().show()

In [ ]:
churn_users = df.filter(df.page=="Cancellation Confirmation").select("userId").dropDuplicates()
churn_users_list = [(row['userId']) for row in churn_users.collect()]
withchurn = df.withColumn("churn", df.userId.isin(churn_users_list))

In [ ]:
withchurn.dropDuplicates(["userId", "gender"]).groupby(["churn", "gender"]).count().sort("churn").show()

In [ ]:
theRatio = withchurn.dropDuplicates(["userId", "gender"]).groupby(["churn", "gender"]).count().sort("churn").toPandas()
sns.barplot(x='churn', y='count', hue='gender', data=theRatio)

In [ ]:
ghour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour)
withchurn = withchurn.withColumn("hour", ghour(withchurn.ts))

gweekday = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%w"))
withchurn = withchurn.withColumn("weekday", gweekday(withchurn.ts))

gday = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).day)
withchurn = withchurn.withColumn("day", gday(withchurn.ts))

In [ ]:
def plotbychurn(time): 
    dfpd = withchurn.filter(withchurn.page == "NextSong").groupby("churn", time).count().orderBy(withchurn[time].cast("float")).toPandas()
    dfpd[time] = pd.to_numeric(dfpd[time])
    dfpd[ dfpd.churn==0].plot.bar(x=time, y='count', color='grey', label='Non churned')
    dfpd[ dfpd.churn==1].plot.bar(x=time, y='count', color='pink', label='Churned')

In [ ]:
plotbychurn("hour")

plotbychurn("weekday")

plotbychurn("day")

In [ ]:
withchurn.dropDuplicates(["userId", "gender"]).groupby(["churn", "gender"]

In [ ]:
print('[{}] Start'.format(datetime.datetime.now()))
print('[{}] Done'.format(datetime.datetime.now()))

In [ ]:
df_new = df.drop('artist', 'auth', 'firstName', 'gender',
                 'itemInSession', 'lastName', 'length', 'location', 'method', 'registration', 'sessionId', 'song', 'avg_hr')

In [ ]:
df_new.toPandas().head()

In [ ]:
index_gender = StringIndexer(inputCol="gender", outputCol='index_gender')
index_page = StringIndexer(inputCol="page", outputCol='index_page')
genderVec = OneHotEncoder(inputCol='index_gender', outputCol='genderVec')
pageVec = OneHotEncoder(inputCol='index_page', outputCol='pageVec')
assembler = VectorAssembler(inputCols=["genderVec", "pageVec", "status"], outputCol="features")
index = StringIndexer(inputCol="churn", outputCol="label")

In [ ]:
train, test = df_new.randomSplit([0.8, 0.2], seed=42)

In [ ]:
Logestic_reg=  LogisticRegression()

pipeline = Pipeline(stages=[Logestic_reg])

In [ ]:
paramGrid = ParamGridBuilder() \
    .addGrid(Logestic_reg.elasticNetParam,[0.0, 0.5, 1.0]) \
    .addGrid(Logestic_reg.regParam,[0.0, 0.01, 0.1]) \
    .build()

crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=3)